<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/carseats.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 성능을 예측합니다.
- 성능이 좋을 것으로 판단된 모델의 성능을 튜닝합니다.
- 튜닝된 모델의 성능을 평가합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [175]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [176]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Carseats.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [178]:
# 데이터 살펴보기
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


**데이터 설명**

- Sales: 각 지역 판매량(단위: 1,000개)
- CompPrice: 경쟁사 가격 (단위: 달러)
- Income: 지역 평균 소득 (단위: 1,000달러)
- Advertising: 각 지역, 회사의 광고 예산 (단위: 1,000달러)
- Population: 지역 인구 수 (단위: 1,000명)
- Price: 자사 지역별 판매 가격 (단위: 달러)
- ShelveLoc: 진열 상태
- Age: 지역 인구의 평균 연령
- Education: 각 지역 교육 수준
- Urban: 도심 지역 여부 (Yes,No)
- US: 매장이 미국에 있는지 여부 (Yes, No)

In [180]:
# 기술통계 확인
data.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [183]:
# target 확인
target = 'Sales'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**2) 가변수화**

In [185]:
# 가변수화 대상: ShelveLoc, Education, Urban, US
dumm_cols = ['ShelveLoc', 'Education', 'Urban', 'US']

# 가변수화
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

,CompPrice,Income,Advertising,Population,Price,Age,ShelveLoc_Good,ShelveLoc_Medium,Education_11,Education_12,Education_13,Education_14,Education_15,Education_16,Education_17,Education_18,Urban_Yes,US_Yes
0,138,73,11,276,120,42,0,0,0,0,0,0,0,0,1,0,1,1
1,111,48,16,260,83,65,1,0,0,0,0,0,0,0,0,0,1,1
2,113,35,10,269,80,59,0,1,0,1,0,0,0,0,0,0,1,1
3,117,100,4,466,97,55,0,1,0,0,0,1,0,0,0,0,1,1
4,141,64,3,340,128,38,0,0,0,0,1,0,0,0,0,0,1,0


**3) 학습용, 평가용 데이터 분리**

In [187]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

**4) 정규화**

In [189]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 다양한 알고리즘으로 모델을 만들고 성능을 확입니다.
- 성능 평가는 mean_absolute_error, r2_score를 사용합니다.

In [191]:
# xgboost 설치
# pip install xgboost

In [192]:
# lightgbm 설치
# pip install lightgbm

- 사용할 알고리즘을 모두 불러옵니다.

In [194]:
# 라이브러리 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, r2_score

**1) Linear Regression**

- Linear Regression 알고리즘으로 모델링하고 성능을 확인합니다.

In [196]:
# 선언하기
model = LinearRegression()

In [197]:
# 학습하기
model.fit(x_train,y_train)

LinearRegression()

In [198]:
# 예측하기
y_pred= model.predict(x_test)

In [199]:

# 5단계: 평가하기
print('MAE:',mean_absolute_error(y_test,y_pred))
print('R2:',r2_score(y_test,y_pred))

MAE: 0.8416098802896127
R2: 0.8657819916636766


**2) KNN**

- KNN 알고리즘으로 모델링하고 성능을 확인합니다.

In [201]:
# 선언하기
model =KNeighborsRegressor(n_neighbors=5)

In [202]:
# 학습하기
model.fit(x_train_s,y_train)

KNeighborsRegressor()

In [203]:
# 예측하기
y_pred = model.predict(x_test_s)

In [204]:
# 평가하기

print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

MAE: 1.8401833333333333
R2: 0.3164099528655834


**3) Decision Tree**

- Decision Tree 알고리즘으로 모델링하고 성능을 확인합니다.

In [206]:
# 선언하기
model = DecisionTreeRegressor(max_depth=5, random_state=1)

In [207]:
# 학습하기
model.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=5, random_state=1)

In [208]:
# 예측하기
y_pred= model.predict(x_test)

In [209]:
# 평가하기

print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:' , r2_score(y_test, y_pred))

MAE: 1.9110944370982414
R2: 0.307084554928293


**4) Random Forest**

- Random Forest 알고리즘으로 모델링하고 성능을 확인합니다.

In [211]:
# 선언하기
model = RandomForestRegressor(max_depth=5, random_state=1)

In [212]:
# 학습하기
model.fit(x_train,y_train)

RandomForestRegressor(max_depth=5, random_state=1)

In [213]:
# 예측하기
y_pred= model.predict(x_test)

In [214]:
# 평가하기

print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:' , r2_score(y_test, y_pred))


MAE: 1.3799522255378287
R2: 0.605314731843458


**5) XGBoost**

- XGBoost 알고리즘으로 모델링하고 성능을 확인합니다.

In [216]:
# 선언하기
model = XGBRegressor(max_depth=5, random_state=1, verbose=-1)

In [217]:
# 학습하기
model.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=1, ...)

In [218]:
# 예측하기
y_pred = model.predict(x_test)

In [219]:
# 평가하기


print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:' , r2_score(y_test, y_pred))

MAE: 1.374096424102783
R2: 0.6010697648596808


**6) LightGBM**

- LightGBM 알고리즘으로 모델링하고 성능을 확인합니다.

In [221]:
# 선언하기
model= LGBMRegressor(max_depth=5, random_state=1, verbose=-1)

In [222]:
# 학습하기
model.fit(x_train, y_train)

LGBMRegressor(max_depth=5, random_state=1, verbose=-1)

In [223]:
# 예측하기
y_pred= model.predict(x_test)

In [224]:
# 평가하기

print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:' , r2_score(y_test, y_pred))


MAE: 1.1793538735127505
R2: 0.6946417907320817
